In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import torch
import open_clip
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from transformers import ViTModel, ViTFeatureExtractor

In [3]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/1. Completed_Data/CanonBeetles.csv"
canon_df = pd.read_csv(file_path)

In [4]:
canon_df["ImageFilePath"] = canon_df["cropped_image_path"].apply(lambda x: f"/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/CANON/{x}")

In [5]:
cols = ['ImageFilePath', 'Genus']
df = canon_df[cols]
df.head(2)

,ImageFilePath,Genus
0,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax
1,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k").to(device)

feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")


In [7]:
def extract_features(image_path):
    
    image = Image.open(image_path).convert("RGB")
    
    inputs = feature_extractor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        features = model(**inputs).last_hidden_state[:, 0, :]

    return features.cpu().numpy()
    

In [8]:
X = np.vstack([extract_features(img) for img in df["ImageFilePath"]])

print(f"Extracted feature shape: {X.shape}")


le = LabelEncoder()

y = le.fit_transform(df["Genus"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()


Extracted feature shape: (1622, 768)


In [9]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [22]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef


seed = 99


models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(64,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


NaiveBayes                | Acc: 88.00% | Prec: 89.20% | Rec: 88.00% | F1: 88.37% | Bal Acc: 86.40% | MCC: 0.7622
LogisticRegression        | Acc: 97.85% | Prec: 97.87% | Rec: 97.85% | F1: 97.79% | Bal Acc: 90.70% | MCC: 0.9547
SVMLinear                 | Acc: 98.46% | Prec: 98.48% | Rec: 98.46% | F1: 98.44% | Bal Acc: 93.63% | MCC: 0.9677
SVMPolynomial             | Acc: 96.00% | Prec: 96.10% | Rec: 96.00% | F1: 95.59% | Bal Acc: 79.10% | MCC: 0.9154
SVMRadialBasis            | Acc: 95.38% | Prec: 95.52% | Rec: 95.38% | F1: 94.60% | Bal Acc: 73.55% | MCC: 0.9022
NearestNeighbor           | Acc: 92.31% | Prec: 91.91% | Rec: 92.31% | F1: 91.59% | Bal Acc: 70.78% | MCC: 0.8353
RandomForest              | Acc: 92.31% | Prec: 88.89% | Rec: 92.31% | F1: 90.56% | Bal Acc: 63.26% | MCC: 0.8350
MLP_Baseline              | Acc: 98.46% | Prec: 98.47% | Rec: 98.46% | F1: 98.44% | Bal Acc: 93.80% | MCC: 0.9677


In [23]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,Genus,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
135,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax
844,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax


In [24]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/18.ViT-linear-probing-genus.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/18.ViT-linear-probing-genus-metrics.csv", index=False)